# MNIST experiments

faisal.qureshi@ontariotechu.ca

In [ ]:
from importlib import reload
import matplotlib.pyplot as plt
import torch
import data
import modelae
import train
import evaluate

# Data

In [ ]:
reload(data)

In [ ]:
dataset = data.set_mnist_data(datafolder='../../datasets', digits=[6])

In [ ]:
img, label = data.view(dataset)
plt.figure(figsize=(5,5))
plt.imshow(img)
plt.title(label);

## Training, validation and test datasets

In [ ]:
training_dataset, validation_dataset, test_dataset = data.split_training_validation_test(dataset, 0.5, 0.2, 0.3)

# Model

In [ ]:
reload(modelae)

In [ ]:
ae = modelae.AutoEncoder()
print(ae)

# Compute environment

In [ ]:
print(f'Cuda is available: {torch.cuda.is_available()}')

for i in range(torch.cuda.device_count()):
    print(f'Device {i}: {torch.cuda.get_device_name(i)}')

# Model training

## Setting up training, validation, and test dataloaders

In [ ]:
training_dataloader, validation_dataloader, test_dataloader = data.set_dataloaders(training_dataset,
                                                                                   validation_dataset,
                                                                                   test_dataset,
                                                                                   batch_size=16,
                                                                                   shuffle=False)

## Training

In [ ]:
reload(train)

In [ ]:
training = train.Training(0)

training.set_loss()
training.set_model(ae)
training.set_optimizer(learning_rate=1e-2, weight_decay=1e-5)
training.set_training_dataloader(training_dataloader)
training.set_validation_dataloader(validation_dataloader)

### Do some training for a few epochs.

In [ ]:
training.train(5, show_loss=True, checkpt_every=-1)

In [ ]:
training.losses

In [ ]:
training.train(10, show_loss=True, checkpt_every=2)

### Now load from checkpoint and continue training.

In [ ]:
training = train.Training(-1)

training.set_loss()
training.set_model(ae)
training.set_optimizer(learning_rate=0, weight_decay=0)
training.set_training_dataloader(training_dataloader)
training.set_validation_dataloader(validation_dataloader)
training.load_checkpoint('./checkpoints/chkpt-0000015.pt')

In [ ]:
print(training.current_epoch)
print(training.losses)

In [ ]:
training.train(5, show_loss=True)

# Inference

In [ ]:
reload(evaluate)

## Evaluating on a single test image

Lets first see how well this autoencoder model does on a single test image.  We will pick an image from our test_dataset

In [ ]:
inference = evaluate.Evaluate(0)
inference.set_loss()
inference.set_model(training.get_model())

In [ ]:
idx = 130
data_item = test_dataset[idx]
input_img = data_item['sample'] 

rec_img, loss, _ = inference.reconstruct_one_image(input_img)

plt.figure(figsize=(10,5))
plt.suptitle('Image reconstruction')
plt.subplot(121)
plt.title('Input image')
plt.imshow(input_img.squeeze())
plt.subplot(122)
plt.title(f'Loss={loss:06.5}')
plt.imshow(rec_img.squeeze());

In [ ]:
test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                              batch_size=1,
                                              shuffle=False)

inference.evaluate(test_dataloader);

In [ ]:
print(loss)

In [ ]:
output.shape

In [ ]:
idx[:3]

In [ ]:
idx[3:7]